In [1]:
from contextlib import contextmanager
from pathlib import Path
import re, os, sys, getopt, time, pickle
import pandas as pd
import numpy as np
import fiona
import rasterio as rio
from rasterio.mask import mask
from rasterio import Affine, MemoryFile
from rasterio.enums import Resampling

In [2]:
def smr_vegsoil(img, endmembers="endmembers.csv") :
    ''' unmix veg. vs soil
    
    To-Do:
        1. set endmembers to be controllable via 
    '''
    
    local_bbs = np.in1d(image, self.bbs)

    w = 1 # set unit sum constraint weight
    D = image
    d = np.reshape(D, [D.shape[0], D.shape[1]*D.shape[2]])
    G = pd.read_csv(endmembers, sep="[,|\t]", header = None).to_numpy()
    wavelength = G[:,0]
    G = G[: ,1:G.shape[1]]

    d = d[local_bbs]
    G = G[local_bbs]

    d_constraint = np.array(w*np.ones(d.shape[1]))
    G_constraint = np.array(w*np.ones(G.shape[1]))

    d = np.vstack([d,d_constraint])
    G = np.vstack([G,G_constraint])

    M_alt = np.linalg.inv(G.transpose().dot(G)).dot(G.transpose().dot(d))
    M = np.reshape(M_alt, [d.shape[0],D.shape[1],D.shape[2]])
    #mixtureResidual_alt = d-(G.dot(np.linalg.inv(G.transpose().dot(G)).dot(G.transpose()))).dot(d)
    #smr = np.reshape(mixtureResidual_alt,[d.shape[0],D.shape[1],D.shape[2]])

    return M

In [3]:
def generate_mask() :

    #masked_image = 

    return masked_image


def apply_mask(si, mask_raster) :
    
    mask = generate_mask(si)
    
    bands = si.shape[0]
    rows = mask.shape[0]
    cols = mask.shape[1]

    masked_bands_matrix = np.empty((band_count,rows,cols,), np.float32)

    for band in range(bands):
        applied_mask = raster[band] * mask
        applied_mask[applied_mask == 0] = np.nan
        masked_bands_matrix[band,:,:] = applied_mask

    return masked_si

In [4]:
def unpickle_rf_model(rfm_pk_filepath) :
    '''unpickle_rf_model loads and unpickles a random-forest model trained
    on spectral residuals and disease incidence points.
    rfm_pj - expected to be a filepath to a pickled (.pkl) random-forest model.

    returns the unpickled random-forest model
    '''
    with open(rfm_pk_filepath, 'rb') as unpickled_model :
        rf_model = pickle.load(unpickled_model)


def clean_wl(src_wls) :
    '''
    '''
    src_wls = src_wls.values()
    wls = [re.findall('\d+\.\d+',wl)[0] for wl in src_wls if len(re.findall('\d+\.\d+',wl)) != 0]
    float_wls = [round(float(wl),6) for wl in wls if wl.replace('.','',1).isdigit()]
    
    return sorted(float_wls)


def remove_bb(wls) :
    '''
    '''
    bbs = [[300,400],[1320,1430],[1800,1960],[2450,2600]]

    np_wls = np.array(wls)
    for bb in bbs :
        np_wls = np_wls[(np_wls <= bb[0]) | (np_wls >= bb[1])]

    return np_wls


def null_nodata_pixels(src_img) :
    band_count, cols, rows = src_img.shape()

    nulled_image = np.empty((band_count,cols,rows), np.float64)

    for band_position in range(band_count) :
        band = spectra[band_position]
        band[band == -9999] = np.nan
        nulled_image[band_position,:,:] = band

    return nulled_image


def mask_imagery(poly_filepath, si_filepath) :
    ''' Clips the spectra according to the boundaries of the polygon.
    '''
    with fiona.open(poly_filepath, "r") as geometries :
        geom = [feature["geometry"] for feature in geometries]

    with rio.open(si_filepath) as src:
        spectra, geog_transform = rio.mask.mask(src, geom, crop=True)
        out_meta = src.meta
        wls = clean_wl(src.tags()) # sorted list of wls; e.g [300.0, ... , 2500.0]
        bbs = remove_bb(wls) # filtered list of wls; e.g. [405.0, ..., 2445]

        out_meta.update({
            "height" : spectra.shape[1],
            "width" : spectra.shape[2],
            "transform" : geog_transform
            })
    
    return spectra, out_meta

In [5]:
def classify_raster(image, meta, outputdir, output_filename="GLRaV") :
    ''' Apply RF model to the clipped and spectrally unmixed aviris image
    '''
    # To-Do
    # 1. Dynamic filenames

    output = ('%s%s_predictions.tif' % (outputdir, output_filename))
    output_classes = ('%s%s_class_probability.tif' % (outputdir,output_filename))

    classEncoder = LabelEncoder()

    colWLHeaders = bbs.astype(str) # missing

    cleanSpectra = image
    b,h,w = cleanSpectra.shape # Band, Height, Width

    # numpy-nd array > df
    local_df = pd.DataFrame(cleanData.reshape([b,-1]).T, columns= colWLHeaders)
    local_df_nn = local_df.dropna()

    local_df_nn['classifications'] = rf_model.predict(local_df_nn[colWLHeaders])

    # Class-Probabilities
    class_probas = rf_model.predict_proba(local_df_nn[colWLHeaders])
    class_count = len(class_probas[1])
    classes = list(map(str,range(1,class_count+1)))

    local_df_nn[classes] = class_probas

    classEncoder.fit(local_df_nn['classifications'])
    classValues = classEncoder.transform(local_df_nn['classifications'])
    local_df_nn['classifications'] = classValues

    local_df_j = local_df.join(local_df_nn[['classifications']+classes])

    local_df_arr = np.array(local_df_j['classifications'])
    output_raster = local_df_arr.reshape((1,h,w))

    output_probs_raster = np.zeros((len(classes),h,w))
    for i,c in enumerate(classes) :
        local_df_probs_arr = np.array(local_df_j[c])
        output_probs_raster[i] = local_df_probs_arr.reshape(1,h,w)

    metadata.update({'count': 1,'drive':'GTiff'})
    with rio.open(output,'w',**metadata) as dest :
        dest.write(output_raster)

    metadata.update({'count':len(classes),'drive':'GTiff'})
    with rio.open(output_classes,'w',**metadata) as dest :
        dest.write(output_probs_raster)

In [6]:
def pipeline(model_filepath, si_filepath, poly_filepath, outputdir) :
    '''
    '''
    # Step 1: unpickle the model
    model = unpickle_rf_model(model_filepath)
    
    # Step 2: Clip AVIRIS-NG Imagery
    vineyard_si, meta = mask_imagery(poly_filepath, si_filepath)
    return vineyard_si, meta
    # Step 3: remove -9999s from Imagery and replace with NA
    #vineyard_si = null_nodata_pixels(vineyard_si)
    # Step 3: calculate spectral mixture residual weights
    
    # Step 4: Generate vegetation mask
    
    # Step 5: Apply vegetation mask to Step 3 output
    
    # Step 6: Apply model to SMR
    

In [ ]:
# use context manager so DatasetReader and MemoryFile get cleaned up automatically
@contextmanager
def inmemory_writer(numpy_arr, meta):
    with MemoryFile() as memfile:
        with memfile.open(**meta) as dataset: # Open as DatasetWriter
            dataset.write(numpy_arr)
            return dataset
        #    del numpy_arr
        #with memfile.open() as dataset: # Reopen as DatasetReader
        #    yield dataset # Note yield not return

In [7]:
data_dir = '/media/ferg/IronWolf_10TB/'
m = 'RFm_smr_H_SyAsy.pk'
si = data_dir + 'AVIRIS_NG/brdf_rfl/ang20200918t210249_rfl_topo_brdf'
poly = data_dir + 'Thesis/data/polys/vineyard_polys/VineyardID_LT_124.geojson'
o = './'

si, out_meta = pipeline(m, si, poly, o)

#with inmemory_writer(si, out_meta) as dataset :
#    print(dataset.read())

/home/ferg/miniconda3/envs/pipelinedev/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ferg/miniconda3/envs/pipelinedev/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ferg/miniconda3/envs/pipelinedev/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [8]:
si.shape

(425, 648, 552)

In [ ]:
out_meta